In [1]:
import pandas as pd

data = {
    "user": [1,1,1,2,2,3,3,4,4],
    "item": ["Movie A","Movie B","Movie C",
             "Movie A","Movie D",
             "Movie B","Movie C",
             "Movie A","Movie D"],
    "rating": [5,3,4,4,5,2,5,3,4]
}

df = pd.DataFrame(data)
df


,user,item,rating
0,1,Movie A,5
1,1,Movie B,3
2,1,Movie C,4
3,2,Movie A,4
4,2,Movie D,5
5,3,Movie B,2
6,3,Movie C,5
7,4,Movie A,3
8,4,Movie D,4


In [2]:
user_item_matrix = df.pivot_table(
    index='user',
    columns='item',
    values='rating'
).fillna(0)

user_item_matrix


item,Movie A,Movie B,Movie C,Movie D
user,,,,
1,5.0,3.0,4.0,0.0
2,4.0,0.0,0.0,5.0
3,0.0,2.0,5.0,0.0
4,3.0,0.0,0.0,4.0


In [3]:
from sklearn.metrics.pairwise import cosine_similarity

user_similarity = cosine_similarity(user_item_matrix)

user_similarity_df = pd.DataFrame(
    user_similarity,
    index=user_item_matrix.index,
    columns=user_item_matrix.index
)

user_similarity_df


user,1,2,3,4
user,,,,
1,1.000000,0.441726,0.682793,0.424264
2,0.441726,1.000000,0.000000,0.999512
3,0.682793,0.000000,1.000000,0.000000
4,0.424264,0.999512,0.000000,1.000000


In [4]:
def recommend_items(user_id, matrix, similarity_df, top_n=2):
    similar_users = similarity_df[user_id].sort_values(ascending=False)[1:]

    weighted_scores = matrix.loc[similar_users.index].T.dot(similar_users)
    recommendations = weighted_scores.sort_values(ascending=False)

    return recommendations.head(top_n)


In [5]:
print("Recommendations for User 1:")
recommend_items(1, user_item_matrix, user_similarity_df)


Recommendations for User 1:


,0
item,
Movie D,3.905687
Movie C,3.413967


### Recommendation Explanation

The recommendation system uses collaborative filtering based on cosine similarity.
It identifies users with similar preferences and recommends items that those users
have rated highly but the target user has not yet interacted with.
